# Compositionality

1. Step 1: Get all the sentences
2. Step 2: Get the probes 
3. Step 3: Test for compositionality ()

In [30]:
import json
import torch

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

MODELS = [
    "Alibaba-NLP/gte-large-en-v1.5",
    "intfloat/multilingual-e5-large",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/all-MiniLM-L6-v2"
]



In [31]:
DATA_PATH = 'https://huggingface.co/datasets/matthieunlp/spatial_geometry/resolve/main/src/data/sentence-embeddings'
RELATIONS_JSON_PATH = '../data/relations.json'

In [32]:
with open(RELATIONS_JSON_PATH, 'r') as f:
    relations = json.load(f)['spatial_relations']

In [33]:
def get_relations_lookup(relations):
    relations_lookup = {}
    for category, category_pairs in relations.items():
        for first, second in category_pairs:
            relations_lookup[first] = {'category': category, 'opposite': second, 'position': 0}
            relations_lookup[second] = {'category': category, 'opposite': first, 'position': 1}
    return relations_lookup

relations_lookup = get_relations_lookup(relations)

In [34]:
def load_embeddings_for_model(model_name, datapoint='relation'):
    import requests
    from io import BytesIO

    embeddings = []
    labels = []
    url = f'{DATA_PATH}/{model_name.replace("/", "_")}.pt'
    response = requests.get(url)
    response.raise_for_status()
    raw_data = torch.load(BytesIO(response.content), weights_only=False)
    
    for data_point in raw_data:
        embeddings.append(data_point['embedding'])
        labels.append(data_point[datapoint])
    return np.array(embeddings), np.array(labels).reshape(-1, 1)

# Probes for all relations

We expect that we will find linear relations, but we also have to isolate the syntaxic elements. 

We train a couple series of probes. 

In [35]:
def train_multitask_probe_for_datapoint(results_dict, models_dict, encoder):
    for model_name in MODELS:
        print(f"Training multitask probe for {model_name} on all datapoints...")
        X, y_relation = load_embeddings_for_model(model_name, datapoint='relation')
        X, y_subject = load_embeddings_for_model(model_name, datapoint='subject')
        X, y_object = load_embeddings_for_model(model_name, datapoint='object')

        # One-hot encode all labels together
        y_combined = np.column_stack([y_relation, y_subject, y_object])  # Stack labels side by side
        y_encoded = encoder.fit_transform(y_combined).todense()
        
        y_encoded = encoder.fit_transform(y).todense()
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)

        y_train = np.asarray(y_train)
        y_test = np.asarray(y_test)

        # Train probe
        clf = MLPClassifier(hidden_layer_sizes=(), 
                            early_stopping=True, 
                            activation='identity')
        clf.fit(X_train, y_train)
        models_dict[model_name] = clf

        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results_dict[model_name] = accuracy
        print(f"Accuracy for {model_name}: {accuracy:.2f}")

In [36]:
def train_probe_for_datapoint(datapoint, results_dict, models_dict, encoder):
    for model_name in MODELS:
        print(f"Training probe for {model_name} on {datapoint}...")
        X, y = load_embeddings_for_model(model_name, datapoint=datapoint)
        y_encoded = encoder.fit_transform(y).todense()
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2)

        y_train = np.asarray(y_train)
        y_test = np.asarray(y_test)

        # Train probe
        clf = MLPClassifier(hidden_layer_sizes=(), early_stopping=True, activation='identity')
        clf.fit(X_train, y_train)
        models_dict[model_name] = clf

        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results_dict[model_name] = accuracy
        print(f"Accuracy for {model_name}: {accuracy:.2f}")


results = {}
models = {}
one_hot_encoder = OneHotEncoder()

subject_results = {}
subject_models = {}
subject_one_hot_encoder = OneHotEncoder()

object_results = {}
object_models = {}
object_one_hot_encoder = OneHotEncoder()

multitask_results = {}
multitask_models = {}
multitask_one_hot_encoder = OneHotEncoder()

train_probe_for_datapoint('relation', results, models, one_hot_encoder)
train_probe_for_datapoint('subject', subject_results, subject_models, subject_one_hot_encoder)
train_probe_for_datapoint('object', object_results, object_models, object_one_hot_encoder)
train_multitask_probe_for_datapoint(multitask_results, multitask_models, multitask_one_hot_encoder)

Training probe for Alibaba-NLP/gte-large-en-v1.5 on relation...
Accuracy for Alibaba-NLP/gte-large-en-v1.5: 1.00
Training probe for intfloat/multilingual-e5-large on relation...
Accuracy for intfloat/multilingual-e5-large: 1.00
Training probe for sentence-transformers/all-mpnet-base-v2 on relation...


/Users/matthieu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy for sentence-transformers/all-mpnet-base-v2: 0.99
Training probe for sentence-transformers/all-MiniLM-L6-v2 on relation...
Accuracy for sentence-transformers/all-MiniLM-L6-v2: 1.00
Training probe for Alibaba-NLP/gte-large-en-v1.5 on subject...
Accuracy for Alibaba-NLP/gte-large-en-v1.5: 1.00
Training probe for intfloat/multilingual-e5-large on subject...
Accuracy for intfloat/multilingual-e5-large: 0.99
Training probe for sentence-transformers/all-mpnet-base-v2 on subject...
Accuracy for sentence-transformers/all-mpnet-base-v2: 1.00
Training probe for sentence-transformers/all-MiniLM-L6-v2 on subject...
Accuracy for sentence-transformers/all-MiniLM-L6-v2: 1.00
Training probe for Alibaba-NLP/gte-large-en-v1.5 on object...
Accuracy for Alibaba-NLP/gte-large-en-v1.5: 0.99
Training probe for intfloat/multilingual-e5-large on object...
Accuracy for intfloat/multilingual-e5-large: 0.99
Training probe for sentence-transformers/all-mpnet-base-v2 on object...


/Users/matthieu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy for sentence-transformers/all-mpnet-base-v2: 0.98
Training probe for sentence-transformers/all-MiniLM-L6-v2 on object...


...

/Users/matthieu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy for intfloat/multilingual-e5-large: 0.99
Training multitask probe for sentence-transformers/all-mpnet-base-v2 on all datapoints...


/Users/matthieu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Accuracy for sentence-transformers/all-mpnet-base-v2: 0.98
Training multitask probe for sentence-transformers/all-MiniLM-L6-v2 on all datapoints...
Accuracy for sentence-transformers/all-MiniLM-L6-v2: 0.97


/Users/matthieu/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [37]:
import pickle

class PickleSaver:
    def __init__(self, data, filename):
        self.data = data
        self.filename = filename

    def save(self):
        with open(self.filename, 'wb') as f:
            pickle.dump(self.data, f)

# Create instances of PickleSaver for each data object
pickle_savers = [
    PickleSaver(results, 'results.pkl'),
    PickleSaver(models, 'models.pkl'),
    PickleSaver(one_hot_encoder, 'one_hot_encoder.pkl'),
    PickleSaver(subject_results, 'subject_results.pkl'),
    PickleSaver(subject_models, 'subject_models.pkl'),
    PickleSaver(subject_one_hot_encoder, 'subject_one_hot_encoder.pkl'),
    PickleSaver(object_results, 'object_results.pkl'),
    PickleSaver(object_models, 'object_models.pkl'),
    PickleSaver(object_one_hot_encoder, 'object_one_hot_encoder.pkl'),
    PickleSaver(multitask_results, 'multitask_results.pkl'),
    PickleSaver(multitask_models, 'multitask_models.pkl'),
    PickleSaver(multitask_one_hot_encoder, 'multitask_one_hot_encoder.pkl')
]

# Save all data using the PickleSaver instances
for saver in pickle_savers:
    saver.save()



# Finding all the elements

We then build a dictionnary with the representation

# Transformation between subject and object

In [48]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

subject_class_representations = {}
for model_name, clf in subject_models.items():
    print(f"Processing subject model: {model_name}")
    weights = clf.coefs_[0]
    class_names = subject_one_hot_encoder.get_feature_names_out()
    subject_class_representations[model_name] = {
        class_name.replace('x0_', ''): weights[:, i]
        for i, class_name in enumerate(class_names)
    }

# For object models
object_class_representations = {}
for model_name, clf in object_models.items():
    print(f"Processing object model: {model_name}")
    weights = clf.coefs_[0]
    class_names = object_one_hot_encoder.get_feature_names_out()
    object_class_representations[model_name] = {
        class_name.replace('x0_', ''): weights[:, i]
        for i, class_name in enumerate(class_names)
    }


Processing subject model: Alibaba-NLP/gte-large-en-v1.5
Processing subject model: intfloat/multilingual-e5-large
Processing subject model: sentence-transformers/all-mpnet-base-v2
Processing subject model: sentence-transformers/all-MiniLM-L6-v2
Processing object model: Alibaba-NLP/gte-large-en-v1.5
Processing object model: intfloat/multilingual-e5-large
Processing object model: sentence-transformers/all-mpnet-base-v2
Processing object model: sentence-transformers/all-MiniLM-L6-v2


In [50]:


merged_class_representations = []

for model_name in subject_class_representations.keys():
    if model_name in object_class_representations:
        merged_class_representations.append({
            "model_name": model_name,
            "subject_embedding": subject_class_representations[model_name],
            "object_embedding": object_class_representations[model_name]
        })

merged_class_representations_df = pd.DataFrame(merged_class_representations)
print(merged_class_representations_df)






                                model_name  \
0            Alibaba-NLP/gte-large-en-v1.5   
1           intfloat/multilingual-e5-large   
2  sentence-transformers/all-mpnet-base-v2   
3   sentence-transformers/all-MiniLM-L6-v2   

                                   subject_embedding  \
0  {'backpack': [0.45930824, 0.13927732, 0.052427...   
1  {'backpack': [1.5536985, -1.935699, -2.3292258...   
2  {'backpack': [0.15356827, 0.8909149, 2.0928144...   
3  {'backpack': [7.621316, -2.8898807, 1.0278848,...   

                                    object_embedding  
0  {'backpack': [-0.4304129, -0.319799, -0.475969...  
1  {'backpack': [-0.9528372, -0.9325819, -0.30154...  
2  {'backpack': [-0.26464918, 1.5674798, -1.01602...  
3  {'backpack': [-14.387648, 0.7471546, 2.85581, ...  


TypeError: expected Tensor as element 0 in argument 0, but got dict

In [55]:
# Create dictionaries to store embeddings for each model and class
subject_embeddings_by_model = {}
object_embeddings_by_model = {}

# Group by model name and class
for item in merged_class_representations:
    model_name = item["model_name"]
    
    # Initialize dicts for new models
    if model_name not in subject_embeddings_by_model:
        subject_embeddings_by_model[model_name] = {}
        object_embeddings_by_model[model_name] = {}
    
    # Store all class representations
    for class_name, embedding in item["subject_embedding"].items():
        if class_name not in subject_embeddings_by_model[model_name]:
            subject_embeddings_by_model[model_name][class_name] = []
        subject_embeddings_by_model[model_name][class_name].append(torch.tensor(embedding))
    
    for class_name, embedding in item["object_embedding"].items():
        if class_name not in object_embeddings_by_model[model_name]:
            object_embeddings_by_model[model_name][class_name] = []
        object_embeddings_by_model[model_name][class_name].append(torch.tensor(embedding))

# Stack tensors for each model and class
subject_stacked = {
    model_name: {
        class_name: torch.stack(tensors).numpy()
        for class_name, tensors in class_dict.items()
    }
    for model_name, class_dict in subject_embeddings_by_model.items()
}

object_stacked = {
    model_name: {
        class_name: torch.stack(tensors).numpy()
        for class_name, tensors in class_dict.items()
    }
    for model_name, class_dict in object_embeddings_by_model.items()
}

# Now you can access embeddings for each model and class:
for model_name in subject_stacked:
    print(f"\nModel: {model_name}")
    for class_name in subject_stacked[model_name]:
        print(f"\nClass: {class_name}")
        print(f"Subject embeddings shape: {subject_stacked[model_name][class_name].shape}")
        print(f"Object embeddings shape: {object_stacked[model_name][class_name].shape}")


Model: Alibaba-NLP/gte-large-en-v1.5

Class: backpack
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: bag
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: ball
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: basket
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: bed
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: bicycle
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: book
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: bottle
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: box
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: brush
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: camera
Subject embeddings shape: (1, 1024)
Object embeddings shape: (1, 1024)

Class: candle
Subject embe

In [75]:

class EmbeddingModelTrainer:
    def __init__(self, subject_embeddings, object_embeddings):
        self.X = np.vstack([arr.squeeze() for arr in subject_embeddings.values()])
        self.y = np.vstack([arr.squeeze() for arr in object_embeddings.values()])
        self.models = {}

    def train_linear_model(self):
        self.models['linear'] = LinearRegression()
        self.models['linear'].fit(self.X, self.y)
        self.models['linear_inverse'] = LinearRegression()
        self.models['linear_inverse'].fit(self.y, self.X)

    def train_ridge_model(self, alpha=1.0):
        self.models['ridge'] = Ridge(alpha=alpha, fit_intercept=True, solver='svd')
        self.models['ridge'].fit(self.X, self.y)
        self.models['ridge_inverse'] = Ridge(alpha=alpha, fit_intercept=True, solver='svd')
        self.models['ridge_inverse'].fit(self.y, self.X)

    def evaluate_model(self, model_name, embeddings, true_embeddings):
        predictions = self.models[model_name].predict(embeddings)
        mse = mean_squared_error(true_embeddings, predictions)
        mae = mean_absolute_error(true_embeddings, predictions)
        r2 = r2_score(true_embeddings, predictions)
        return predictions, mse, mae, r2

    def print_evaluation(self, model_name, X_test, y_true, description):
        # Convert test data to proper format
        X_test = np.vstack([arr.squeeze() for arr in X_test.values()])
        y_true = np.vstack([arr.squeeze() for arr in y_true.values()])
        
        # Make predictions
        y_pred = self.models[model_name].predict(X_test)
        
        # Calculate MSE
        mse = mean_squared_error(y_true, y_pred)
        print(f"\n{description}")
        print(f"Mean Squared Error for {model_name}: {mse:.6f}")

# Usage
for model_name in subject_stacked:
    subject_embeddings = subject_stacked[model_name]
    object_embeddings = object_stacked[model_name]
    trainer = EmbeddingModelTrainer(subject_embeddings, object_embeddings)
    trainer.train_linear_model()
    trainer.train_ridge_model()

    trainer.print_evaluation('linear', subject_embeddings, object_embeddings, "Predicted object embeddings")
    trainer.print_evaluation('ridge', subject_embeddings, object_embeddings, "Predicted object embeddings with regularization")
    # trainer.print_evaluation('linear_inverse', object_embeddings, subject_embeddings, "Predicted subject embeddings")
    # trainer.print_evaluation('ridge_inverse', trainer.models['ridge'].predict(subject_embeddings), subject_embeddings, "Predicted subject embeddings with regularization inverse")



Predicted object embeddings
Mean Squared Error for linear: 0.000000

Predicted object embeddings with regularization
Mean Squared Error for ridge: 0.000017

Predicted object embeddings
Mean Squared Error for linear: 0.000000

Predicted object embeddings with regularization
Mean Squared Error for ridge: 0.000000

Predicted object embeddings
Mean Squared Error for linear: 0.000000

Predicted object embeddings with regularization
Mean Squared Error for ridge: 0.000001

Predicted object embeddings
Mean Squared Error for linear: 0.000000

Predicted object embeddings with regularization
Mean Squared Error for ridge: 0.000001


In [156]:
merged_class_representations_df['subject_embedding'][0]

{'backpack': array([ 0.45930824,  0.13927732,  0.05242767, ...,  0.04325312,
        -0.23257335,  0.133842  ], dtype=float32),
 'bag': array([ 0.24440594, -0.17168754, -0.88721865, ..., -0.5350134 ,
        -0.15706661, -0.334713  ], dtype=float32),
 'ball': array([ 0.23578249,  0.13245244, -0.31964183, ...,  0.08768918,
        -0.08186665,  0.19385338], dtype=float32),
 'basket': array([ 0.33961737, -0.18598752, -0.02805528, ..., -0.15965174,
         0.12199956, -0.24671899], dtype=float32),
 'bed': array([ 0.1752361 , -0.23391986,  0.02540315, ..., -0.32959452,
        -0.1558452 ,  0.13773447], dtype=float32),
 'bicycle': array([-0.5912732 ,  0.18265815, -0.13700363, ..., -0.2944202 ,
         0.20219561,  0.10965832], dtype=float32),
 'book': array([ 0.6345695 ,  0.14126243, -0.0973493 , ..., -0.57434565,
         0.11965234,  0.13884461], dtype=float32),
 'bottle': array([ 0.5292081 ,  0.26253563,  0.1514274 , ...,  0.09160279,
         0.02351419, -0.04712278], dtype=float32),

# Transformation between sentences

If such is the case, we could potentially check if we could learn a mapping to fully invert the sentence. 

In [160]:
from sklearn.metrics import mean_squared_error

mapping_results = {}
mapping_models = {}

def train_linear_mapping_on_embeddings(data, results_dict, models_dict):

    X = np.stack(data['embeddings'].apply(lambda x: x[0]).values)
    y = np.stack(data['embeddings'].apply(lambda x: x[1]).values)
    print(X.shape, y.shape)


    linear_model = LinearRegression()
    linear_model.fit(X, y)
    model_name = f"linear_model"
    mapping_models[model_name] = linear_model

    y_pred = linear_model.predict(X)
    mse = mean_squared_error(y, y_pred)
    mapping_results[model_name] = mse
    print(f"MSE for {model_name}: {mse:.2f}")


In [161]:

results_dict = {}
models_dict = {}

train_linear_mapping_on_embeddings(data['Alibaba-NLP_gte-large-en-v1.5.pt'], results_dict, models_dict)

(71050, 1024) (71050, 1024)
MSE for linear_model: 0.02


In [ ]:
import numpy as np
def analyze_transformation(model, X, y):
    W = model.coef_  # Get transformation matrix
    
    # 1. Basic Shape Info
    print(f"Transformation matrix shape: {W.shape}")
    print(f"Number of parameters: {W.size:,}")
    
    # 2. Check prediction quality
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    print(f"\nPrediction Quality:")
    print(f"MSE: {mse:.6f}")
    print(f"R² score: {r2:.6f}")
    
    # 3. Matrix Properties
    print(f"\nMatrix Properties:")
    print(f"Mean coefficient value: {np.mean(W):.6f}")
    print(f"Std of coefficients: {np.std(W):.6f}")
    print(f"% of zeros: {(np.abs(W) < 1e-10).mean():.2%}")
    
    # 4. Check if mostly diagonal
    diagonal_strength = np.abs(np.diag(W)).mean()
    off_diagonal_strength = np.abs(W - np.diag(np.diag(W))).mean()
    print(f"\nDiagonal vs Off-diagonal:")
    print(f"Diagonal strength: {diagonal_strength:.6f}")
    print(f"Off-diagonal strength: {off_diagonal_strength:.6f}")
    
    # 5. Rank analysis
    rank = np.linalg.matrix_rank(W)
    print(f"\nRank Analysis:")
    print(f"Matrix rank: {rank} / {W.shape[0]}")
    
    # 6. Top singular values
    U, s, Vh = np.linalg.svd(W)
    print(f"\nTop 5 singular values:")
    print(s[:5])

# Use it on your model
analyze_transformation(
    mapping_models['linear_model'],
    np.stack(data['Alibaba-NLP_gte-large-en-v1.5.pt']['embeddings'].apply(lambda x: x[0]).values),
    np.stack(data['Alibaba-NLP_gte-large-en-v1.5.pt']['embeddings'].apply(lambda x: x[1]).values)
)

# Optional: Check for overfitting
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

print("\nGeneralization Check:")
print(f"Train MSE: {mean_squared_error(y_train, model.predict(X_train)):.6f}")
print(f"Test MSE: {mean_squared_error(y_test, model.predict(X_test)):.6f}")

# Check if it's mostly diagonal
diagonal_strength = np.abs(np.diag(mapping_models['linear_model'].coef_)).mean()
off_diagonal_strength = np.abs(mapping_models['linear_model'].coef_ - np.diag(np.diag(mapping_models['linear_model'].coef_))).mean()
print(f"Diagonal vs off-diagonal strength: {diagonal_strength:.3f} vs {off_diagonal_strength:.3f}")

# Check the rank
rank = np.linalg.matrix_rank(mapping_models['linear_model'].coef_)
print(f"Rank of transformation: {rank} / 1024")



Transformation matrix shape: (1024, 1024)
Number of parameters: 1,048,576


We obtain a linear mapping between the embeddings of the subject and object. 

# Syntax & Transformation 

We have a representation for both could take the representation of subject and object and asess if there is any change. 

# Compositionality 

We now have a representation for all the words and all the relations, we have to create this mapping. We can show that it extends outside. It is already very clear that we can decompose the representation into specific words. But also the position of the subject. 

How would we prove full compositionality ? 


We can now be confident in a form  of compositionality. The transformation from object to subject can be used to build sentences. 